<a href="https://colab.research.google.com/github/yubin8773/2021-deep-learning/blob/main/%EB%B0%B1%EC%B0%AC%ED%98%B8/Covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, shutil
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers

train_dir='/content/drive/MyDrive/dataset/Covid19-dataset/train'
test_dir='/content/drive/MyDrive/dataset/Covid19-dataset/test'

In [2]:
conv_base= VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(224,224,3))

58900480/58889256 [==============================] - 0s 0us/step


In [3]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
model=models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               6422784   
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 21,138,243
Trainable params: 21,138,243
Non-trainable params: 0
_________________________________________________________________


In [6]:
print('conv_base를 동결하기 전 훈련되는 가중치의 수', len(model.trainable_weights))
conv_base.trainable=False
print('동결 후 훈련되는 가중치 수', len(model.trainable_weights))

conv_base를 동결하기 전 훈련되는 가중치의 수 30
동결 후 훈련되는 가중치 수 4


In [7]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   horizontal_flip=True, 
                                   shear_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)


In [8]:
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/dataset/Covid19-dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 251 images belonging to 3 classes.


In [9]:
validation_generator = test_datagen.flow_from_directory('/content/drive/MyDrive/dataset/Covid19-dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 66 images belonging to 3 classes.


In [10]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break

배치 데이터 크기: (32, 224, 224, 3)
배치 레이블 크기: (32, 3)


In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [12]:
history = model.fit_generator(
  train_generator,
  validation_data=validation_generator,
  epochs=7,
  steps_per_epoch=len(train_generator),
  validation_steps=len(validation_generator)
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/7
8/8 [==============================] - 188s 19s/step - loss: 1.4846 - accuracy: 0.4741 - val_loss: 0.6438 - val_accuracy: 0.6970
Epoch 2/7
8/8 [==============================] - 13s 2s/step - loss: 0.4255 - accuracy: 0.8685 - val_loss: 0.3868 - val_accuracy: 0.8636
Epoch 3/7
8/8 [==============================] - 13s 2s/step - loss: 0.3117 - accuracy: 0.9044 - val_loss: 0.3768 - val_accuracy: 0.7879
Epoch 4/7
8/8 [==============================] - 13s 2s/step - loss: 0.1925 - accuracy: 0.9363 - val_loss: 0.2632 - val_accuracy: 0.9394
Epoch 5/7
8/8 [==============================] - 13s 2s/step - loss: 0.2050 - accuracy: 0.9363 - val_loss: 0.2432 - val_accuracy: 0.9394
Epoch 6/7
8/8 [==============================] - 13s 2s/step - loss: 0.1173 - accuracy: 0.9721 - val_loss: 0.2332 - val_accuracy: 0.9242
Epoch 7/7
8/8 [==============================] - 13s 2s/step - loss: 0.1151 - accuracy: 0.9681 - val_loss: 0.2972 - val_accuracy: 0.8182
